In [3]:
import os
import pandas as pd
import gensim.models.word2vec as w2v
from gensim.corpora.textcorpus import TextCorpus
import re
from datetime import datetime
import _pickle as pickle

C:\Users\Ayush\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
def get_chunks(file_size):
    chunk_start = 0
    chunk_size = 0x20000
    while chunk_start + chunk_size < file_size:
        yield(chunk_start, chunk_size)
        chunk_start += chunk_size
    final_chunk_size = file_size - chunk_start
    yield(chunk_start, final_chunk_size)

def read_file_chunked(file_path):
    with open(file_path,encoding='utf-8') as file_:
        file_size = os.path.getsize(file_path)
        print(file_size)
        print('File size: {}'.format(file_size))
        progress = 0

        for chunk_start, chunk_size in get_chunks(file_size):

            file_chunk = file_.read(chunk_size)
#             print((file_chunk))
            f.write(file_chunk)
            progress += len(file_chunk)
            if(progress>=file_size/2):
                break
#             print('{0} of {1} bytes read'.format(progress, file_size))
#             print('In Progress...')
        print('Completed')

f = open('prep.tsv', 'a+', encoding='utf-8')
read_file_chunked('../Dataset/dataset.tsv')

2529193595
File size: 2529193595
Completed


In [5]:
f = open('prep.tsv',encoding='utf-8')
f1 = open('newprep.tsv','w',encoding='utf-8')
for x in range(4781164):
    s = f.readline()
    l = s.split('\t')
    f1.write(l[0][5:]+'\t'+l[1]+'\t'+l[-1])
f.close()
f1.close() 

In [6]:
def timestamp(time):
    arr=re.split('-|T|Z|:',time)
    date_time = arr[0]+arr[1]+arr[2]+arr[3]+arr[4]+arr[5]
#     print(date_time)
    return date_time

In [7]:
columns=['UserID', 'TimeStamp', 'Song'] 
data = pd.read_csv('newprep.tsv',delimiter='\t',encoding='utf-8', names=columns,header=None)
data.head()
print(len(data))


data.sort_values(by=['UserID','TimeStamp'],ascending=[True,True],inplace=True)
MPS = {}
idnum = data.iloc[0]['UserID']
song = data.iloc[0]['Song']
ts = data.iloc[0]['TimeStamp']
l = [song]
for i in range(1,len(data)):
    print(i)
    newid = data.iloc[i]['UserID']
    tsnew = data.iloc[i]['TimeStamp']
#     print(tsnew)
    newsong = data.iloc[i]['Song']
    if newid == idnum:
        ts1 = timestamp(data.iloc[i]['TimeStamp'])
        ts2 = timestamp(data.iloc[i-1]['TimeStamp'])
        d = pd.Timedelta(datetime.strptime(ts1, '%Y%m%d%H%M%S') - datetime.strptime(ts2, '%Y%m%d%H%M%S')).seconds
        if d < 800:
            l.append(data.iloc[i]['Song'])
        else:
            if idnum in MPS.keys():
                MPS[idnum].append(l)
            else:
                MPS[idnum] = [l]
            l = []
            l.append(data.iloc[i]['Song'])
    else:
        if idnum in MPS.keys():
            MPS[idnum].append(l)
        else:
            MPS[idnum] = [l]
        idnum = newid
        l = []
        l.append(data.iloc[i]['Song'])
if l not in MPS[idnum]:
    if idnum in MPS.keys():
        MPS[idnum].append(l)
    else:
        MPS[idnum] = [l]
for user in MPS:
    for ele in MPS[user]:
        if len(ele) < 10:
            MPS[user].remove(ele)

,UserID,TimeStamp,Song
0,1,2009-05-04T23:08:57Z,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,1,2009-05-04T13:54:10Z,Composition 0919 (Live_2009_4_15)
2,1,2009-05-04T13:52:04Z,Mc2 (Live_2009_4_15)
3,1,2009-05-04T13:42:52Z,Hibari (Live_2009_4_15)
4,1,2009-05-04T13:42:11Z,Mc1 (Live_2009_4_15)


In [9]:
f = open("MPS.txt","w",encoding='utf-8')
f.write(str(MPS))
f.close()

pickle_out = open("dict.pickle","wb")
pickle.dump(MPS, pickle_out)
pickle_out.close()

In [86]:
corpus = []
# f = open('MPS1.txt','r',encoding='utf-8')
for i in range(len(MPS)):
    l = MPS[i+1]
#     print(len(l))
    for j in range(len(l)):
        if len(l[j]) > 10:
            corpus.append(l[j])
print(len(corpus))

125181


In [91]:
song2vec = w2v.Word2Vec(sg=1,seed=1,size=100,min_count=10,window=5)
song2vec.build_vocab(corpus)
# # Preprocess the songs that contain unprintable unicode characters
# vocabulary1=list(song2vec.wv.vocab)
# print(len(vocabulary1))
# vocabulary1=list(song2vec.wv.vocab)
# vocabulary1=[x.encode('UTF8') for x in vocabulary1]
# vocabulary=[]
# for word in vocabulary1:
#     if re.match('[A-Za-z0-9\'\?\&\)\(\+\!\-\*\.\, ]+$',word):
#         vocabulary.append(word)

# vocabulary.sort()
# print(vocabulary)
# print(len(vocabulary))

In [1]:
song2vec.save("song2vec.model")

NameError: name 'song2vec' is not defined

In [92]:
song2vec.train(corpus,total_examples=len(corpus),epochs=10)

(31229520, 39464000)

In [ ]:
print(song2vec.most_similar(u'The Boy Looked At Johnny',topn=10))

In [ ]:
print song2vec.most_similar('Gang Of Gin',topn=10)

In [1]:
f = open('MPS.txt','r',encoding='utf-8')

In [2]:
d = {}
with open('MPS.txt','r',encoding='utf-8') as f:
for line in f:
    (key, val) = line.split()
    d[int(key)] = val
print(d)

<_io.TextIOWrapper name='MPS.txt' mode='r' encoding='utf-8'>
